## Load data

In [101]:
import pandas as pd
import ast
import numpy as np

# Load in data
admissions = 'tedsa_puf_2019.csv'
df = pd.read_csv(f'../../Downloads/{admissions}')

## Compare SUB1 groups with DSMCRIT

In [102]:
sub1 = df['SUB1']
for i in [1,2,3,4,8,9,10,11,12,13,14,15,16,17,18,19]:
    sub1 = sub1.replace(i, 'Other')
# sub1 = sub1.replace(-9, np.nan)

dsmcrit = df['DSMCRIT']
for i in [1,2,3,4,6,7,8,9,10,11,13,14,15,16,17,18,19]:
    dsmcrit = dsmcrit.replace(i, 'Other')
# dsmcrit = dsmcrit.replace(-9, np.nan)

In [103]:
df4 = pd.DataFrame()
df4['SUB1'] = sub1
df4['DSMCRIT'] = dsmcrit

df4.value_counts()

print('See "SUB1 DSMCRIT overlap.xlsx" file for a better breakdown')

See "SUB1 DSMCRIT overlap.xlsx" file for a better breakdown


## Filter out select rows and columns

In [104]:
# Drop defined columns
columns_to_drop = ['ADMYR', 'CASEID', 'DETNLF', 'DETCRIM']
df = df.drop(columns=columns_to_drop)
print(f'Dropped {len(columns_to_drop)} columns ({len(df.columns)} remain)')
old_rows = len(df)

# Drop values where dependent variable is unknown
df = df[df['METHUSE'] != -9]

# Drop Puerto Rico
df = df[df['STFIPS'] != 72]

# Only keep patients admitted with self-described use of an opioid as their primary substance use (i.e., SUB1 = 5, 6, or 7)
df = df[df['SUB1'].between(5, 7)]
new_rows = len(df)
percent_change = round(100*(old_rows-new_rows)/old_rows, 1)
print(f'Dropped {old_rows-new_rows} observations or {percent_change}% of the data ({new_rows} rows remain)')

df = df.reset_index(drop='index')

Dropped 4 columns (58 remain)
Dropped 1341403 observations or 71.9% of the data (522964 rows remain)


In [105]:
df

,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,NOPRIOR,PSOURCE,ARRESTS,EMPLOY,METHUSE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2,-9,2,1,7,3,1,0,3,2,...,0,0,0,0,0,0,9,4,1,3
1,2,-9,3,1,7,2,3,0,4,1,...,0,0,0,0,0,0,9,4,1,2
2,2,-9,1,2,7,5,2,0,4,1,...,0,0,0,0,0,0,9,4,1,2
3,2,-9,3,1,7,3,3,0,4,1,...,0,0,0,0,0,0,9,4,1,2
4,2,-9,3,1,7,1,2,0,2,1,...,0,0,0,0,0,0,9,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522959,56,-9,-9,2,6,0,6,0,3,2,...,0,0,0,0,0,0,8,4,0,2
522960,56,-9,-9,2,6,2,7,0,3,2,...,0,0,0,0,0,0,8,4,0,2
522961,56,-9,3,1,7,3,2,0,3,2,...,0,0,1,0,0,0,8,4,0,3
522962,56,-9,1,1,7,0,7,0,4,2,...,0,0,0,0,0,0,8,4,0,2


## Make dataset human-readable

In [106]:
# Load in variable dictionary
with open('variable_dictionary.txt') as file:
    variable_dict_string = file.read()
    variable_dict = ast.literal_eval(variable_dict_string)

# Rename entries in column according to dictionary
df2 = df.copy()
for col, col_dict in variable_dict.items():
    for old_value, new_value in variable_dict[col].items():
        df2[col] = df2[col].replace(old_value, new_value)

# Rename "-9" values as "Unknown"
for col in df2.columns:
    df2[col] = df2[col].replace(-9, 'Unknown')

In [107]:
df2

,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,NOPRIOR,PSOURCE,ARRESTS,EMPLOY,METHUSE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,AK,Unknown,Grade9To11,NeverMarried,AmbulatoryNonIntensiveOutpatient,3PriorTreatments,Individual,0Arrest,Unemployed,NoMethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Pacific,West,IDU,Alcohol&Drugs
1,AK,Unknown,Grade12OrGED,NeverMarried,AmbulatoryNonIntensiveOutpatient,2PriorTreatments,OtherHealthCareProvider,0Arrest,NotInLaborForce,MethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Pacific,West,IDU,OtherDrugs
2,AK,Unknown,GradeOrLess,NowMarried,AmbulatoryNonIntensiveOutpatient,5PlusPriorTreatments,DrugCareProvider,0Arrest,NotInLaborForce,MethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Pacific,West,IDU,OtherDrugs
3,AK,Unknown,Grade12OrGED,NeverMarried,AmbulatoryNonIntensiveOutpatient,3PriorTreatments,OtherHealthCareProvider,0Arrest,NotInLaborForce,MethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Pacific,West,IDU,OtherDrugs
4,AK,Unknown,Grade12OrGED,NeverMarried,AmbulatoryNonIntensiveOutpatient,1PriorTreatments,DrugCareProvider,0Arrest,PartTime,MethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Pacific,West,IDU,OtherDrugs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522959,WY,Unknown,Unknown,NowMarried,AmbulatoryIntensiveOutpatient,0PriorTreatments,OtherReferral,0Arrest,Unemployed,NoMethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Mountain,West,NoIDU,OtherDrugs
522960,WY,Unknown,Unknown,NowMarried,AmbulatoryIntensiveOutpatient,2PriorTreatments,CourtReferral,0Arrest,Unemployed,NoMethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Mountain,West,NoIDU,OtherDrugs
522961,WY,Unknown,Grade12OrGED,NeverMarried,AmbulatoryNonIntensiveOutpatient,3PriorTreatments,DrugCareProvider,0Arrest,Unemployed,NoMethUse,...,NotReported,NotReported,Reported,NotReported,NotReported,NotReported,Mountain,West,NoIDU,Alcohol&Drugs
522962,WY,Unknown,GradeOrLess,NeverMarried,AmbulatoryNonIntensiveOutpatient,0PriorTreatments,CourtReferral,0Arrest,NotInLaborForce,NoMethUse,...,NotReported,NotReported,NotReported,NotReported,NotReported,NotReported,Mountain,West,NoIDU,OtherDrugs


## Make machine-readable dataset
todo clean up this section
fix data leakage

In [ ]:
# Remove dependent variable
methuse = df2['METHUSE']
df3 = df2.drop(columns=['METHUSE'])

# Convert categorical variables to dummy variables
df3 = pd.get_dummies(df2)
df3

In [119]:
methuse = methuse.replace('MethUse', 1)
methuse = methuse.replace('NoMethUse', 0)
methuse

0         0
1         1
2         1
3         1
4         1
         ..
522959    0
522960    0
522961    0
522962    0
522963    0
Name: METHUSE, Length: 522964, dtype: int64

In [120]:
from sklearn.model_selection import train_test_split

# Add intercept
df3.insert(0, 'Intercept', 1)
X = df3
y = methuse

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [122]:
# Load modules and data
import statsmodels
import statsmodels.api as sm

# Fit and summarize OLS model
model = sm.OLS(y_train, X_train)
results = model.fit()

y_hat = results.predict(X_test)
mse = statsmodels.tools.eval_measures.mse(y_test, y_hat)

print('MSE:', mse)
print(results.summary())

MSE: 5.971537135764488e-14
                            OLS Regression Results                            
Dep. Variable:                METHUSE   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.855e+15
Date:                Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                        21:09:57   Log-Likelihood:             5.0534e+06
No. Observations:              366074   AIC:                        -1.011e+07
Df Residuals:                  365558   BIC:                        -1.010e+07
Df Model:                         515                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------